<a href="https://colab.research.google.com/github/fjadidi2001/Artificial_Intelligence_Learning/blob/master/telematics_syn_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify file path
file_path = '/content/drive/My Drive/Dissertation/telematics_syn.csv'

# Import pandas (assuming you want to use it to read the CSV)
import pandas as pd

# Read the CSV file
df = pd.read_csv(file_path)

print(df.shape)  # Should print (100000, 52)

print(df.head())  # Example: print the first few rows of the dataframe


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(100000, 52)
   Duration  Insured.age Insured.sex  Car.age  Marital  Car.use  Credit.score  \
0       366           45        Male       -1  Married  Commute         609.0   
1       182           44      Female        3  Married  Commute         575.0   
2       184           48      Female        6  Married  Commute         847.0   
3       183           71        Male        6  Married  Private         842.0   
4       183           84        Male       10  Married  Private         856.0   

  Region  Annual.miles.drive  Years.noclaims  ...  Left.turn.intensity10  \
0  Urban             6213.71              25  ...                    1.0   
1  Urban            12427.42              20  ...                   58.0   
2  Urban            12427.42              14  ...                    0.0   
3  Urban             6213.71              43  ...                  

#step 2

## pre1

In [15]:
# Encoding categorical columns using one-hot encoding
categorical_cols = ['Marital', 'Insured.sex', 'Car.use', 'Region', 'Territory']
df = pd.get_dummies(df, columns=categorical_cols)

# Creating the ClaimYN variable
df['ClaimYN'] = df['NB_Claim'].apply(lambda x: 1 if x > 0 else 0)

# Saving the preprocessed data to a new file
preprocessed_file_path = '/content/drive/My Drive/pre_telematics_syn.csv'
df.to_csv(preprocessed_file_path, index=False)
print(df.shape)  # Should print (100000, number of columns after encoding)


(100000, 113)


## pre2

In [17]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Feature columns (excluding response columns)
feature_cols = [col for col in df.columns if col not in ['NB_Claim', 'AMT_Claim', 'ClaimYN']]

# Applying StandardScaler and MinMaxScaler
scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

df[feature_cols] = scaler.fit_transform(df[feature_cols])
df[feature_cols] = minmax_scaler.fit_transform(df[feature_cols])

# Save the preprocessed data to a new file
preprocessed_file_path_scaled = '/content/drive/My Drive/pre_telematics_syn_scaled.csv'
df.to_csv(preprocessed_file_path_scaled, index=False)
print(df.shape)  # Should print (100000, number of columns after scaling and normalization)


(100000, 113)


## pre3

In [19]:
# Advanced feature engineering steps, if any
# For example, aggregating harsh driving events:
# Check if the columns exist before using them
if all(col in df.columns for col in ['Accel.xx.miles', 'Brake.xx.miles', 'Left.turn.intensityxx', 'Right.turn.intensityxx']):
    df['HarshDriving'] = df['Accel.xx.miles'] + df['Brake.xx.miles'] + df['Left.turn.intensityxx'] + df['Right.turn.intensityxx']
else:
    print("Warning: One or more columns required for 'HarshDriving' calculation are missing.")

# Save the fully preprocessed data to a new file
preprocessed_file_path_final = '/content/drive/My Drive/pre_telematics_syn_advanced.csv'
df.to_csv(preprocessed_file_path_final, index=False)
print(df.shape)  # Should print (100000, number of columns after feature engineering)

(100000, 113)


In [20]:
from sklearn.model_selection import train_test_split

# Define the response and feature columns
response_cols = ['NB_Claim', 'AMT_Claim', 'ClaimYN']
feature_cols = [col for col in df.columns if col not in response_cols]

# Split into 80% train and 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Further split train into train and validation
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

print(train_df.shape, val_df.shape, test_df.shape)


(64000, 113) (16000, 113) (20000, 113)


In [10]:
!pip install pytorch-tabnet

In [21]:
# Install the library
!pip install pytorch-tabnet

# Import required libraries
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np

# Prepare the data for TabNet
X_train = train_df[feature_cols].values
y_train = train_df['ClaimYN'].values
X_val = val_df[feature_cols].values
y_val = val_df['ClaimYN'].values
X_test = test_df[feature_cols].values
y_test = test_df['ClaimYN'].values

# Create and train the TabNet model
clf = TabNetClassifier()

clf.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'val'],
    eval_metric=['accuracy'],
    max_epochs=50,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.64125 | train_accuracy: 0.19758 | val_accuracy: 0.20319 |  0:00:08s
epoch 1  | loss: 0.57605 | train_accuracy: 0.13955 | val_accuracy: 0.14269 |  0:00:18s
epoch 2  | loss: 0.56285 | train_accuracy: 0.22433 | val_accuracy: 0.22725 |  0:00:27s
epoch 3  | loss: 0.55817 | train_accuracy: 0.3637  | val_accuracy: 0.36531 |  0:00:36s
epoch 4  | loss: 0.55079 | train_accuracy: 0.57991 | val_accuracy: 0.57969 |  0:00:45s
epoch 5  | loss: 0.54498 | train_accuracy: 0.52409 | val_accuracy: 0.525   |  0:00:55s
epoch 6  | loss: 0.5384  | train_accuracy: 0.62292 | val_accuracy: 0.624   |  0:01:03s
epoch 7  | loss: 0.53232 | train_accuracy: 0.63564 | val_accuracy: 0.63525 |  0:01:13s
epoch 8  | loss: 0.53003 | train_accuracy: 0.63997 | val_accuracy: 0.63694 |  0:01:23s
epoch 9  | loss: 0.52534 | train_accuracy: 0.67819 | val_accuracy: 0.67494 |  0:01:31s
epoch 10 | loss: 0.52049 | train_accuracy: 0.67628 | val_accuracy: 0.67094 |  0:01:41s
epoch 11 | loss: 0.51506 | train_accuracy: 

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
